# Load

In [9]:
import os
import cv2
import numpy as np

In [15]:
os.path.join(path2root, subdir_list[0], subdir_list[0]+'-mask.tif')

'/data/CellBinDB/10×Genomics_DAPI/CytAssist_FFPE_Human_Skin_Melanoma_tissue_image_3089/CytAssist_FFPE_Human_Skin_Melanoma_tissue_image_3089-mask.tif'

In [31]:
# Import test mask and image
path2root = '/data/CellBinDB/10×Genomics_DAPI'
subdir_list = os.listdir(os.path.join(path2root))
mask = cv2.imread(os.path.join(path2root, subdir_list[0], subdir_list[0]+'-mask.tif'), cv2.IMREAD_GRAYSCALE)
instancemask= cv2.imread(os.path.join(path2root, subdir_list[0], subdir_list[0]+'-instancemask.tif'), cv2.IMREAD_GRAYSCALE)
image = cv2.imread(os.path.join(path2root, subdir_list[0], subdir_list[0]+'-img.tif'), cv2.IMREAD_GRAYSCALE)

# Convert segmentation to instance
# image = image.astype(np.float32) / 255.0
num_labels, instance_mask = cv2.connectedComponents(mask)
bboxes = []
for label in range(1, num_labels):  # Exclude background (label 0)
    ys, xs = np.where(instance_mask == label)
    if len(xs) == 0 or len(ys) == 0:
        continue
    x_min, y_min, x_max, y_max = min(xs), min(ys), max(xs), max(ys)
    bboxes.append([x_min, y_min, x_max, y_max])

for bbox in bboxes:
    x_min, y_min, x_max, y_max = map(int, bbox)
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
cv2.imwrite('/home/yec23006/projects/research/merfish/FasterRCNN/Result/generated_instance_mask.png', image)

True

# Anchors

In [1]:
import torch
import torchvision.models as models

# Load the pre-trained MobileNet V2 model
model = models.mobilenet_v2(pretrained=True)

# Access the model's configuration
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

/home/yec23006/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yec23006/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
